In [6]:
import json
import decimal


In [7]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)


In [20]:
import matplotlib.pyplot as plt
import boto3
from boto3.dynamodb.conditions import Key

inst = "CL"
db = boto3.resource('dynamodb', region_name='us-east-1',
                    endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
table = db.Table("mktData")
j = table.scan() 

In [21]:

print(j["Items"])